In [1]:
# import the necessary packages
import numpy as np
import imutils
import cv2
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imutils import paths

In [2]:
#Functions
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

def extract_color_histogram(image, bins=(8, 8, 8)):
	# extract a 3D color histogram from the HSV color space using
	# the supplied number of `bins` per channel
	hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
	hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
		[0, 180, 0, 256, 0, 256])
	# handle normalizing the histogram if we are using OpenCV 2.4.X
	if imutils.is_cv2():
		hist = cv2.normalize(hist)
	# otherwise, perform "in place" normalization in OpenCV 3
	else:
		cv2.normalize(hist, hist)
	# return the flattened histogram as the feature vector
	return hist.flatten()

In [3]:
#Images
# grab the list of images that we'll be describing
print("[INFO] describing images...")
imagePaths = list(paths.list_images("./train_vega"))
print (imagePaths)
# initialize the raw pixel intensities matrix, the features matrix,
# and labels list
rawImages = []
features = []
labels = []

[INFO] describing images...
['./train_vega\\Ankh.01.jpg', './train_vega\\Ankh.02.jpg', './train_vega\\Ankh.03.jpg', './train_vega\\Ankh.04.jpg', './train_vega\\Ankh.05.jpg', './train_vega\\Ankh.06.jpg', './train_vega\\Ankh.07.jpg', './train_vega\\Ankh.08.jpg', './train_vega\\Ankh.09.jpg', './train_vega\\Ankh.10.jpg', './train_vega\\Ankh.11.jpg', './train_vega\\Ankh.12.jpg', './train_vega\\Ankh.13.jpg', './train_vega\\Ankh.14.jpg', './train_vega\\Ankh.15.jpg', './train_vega\\Ankh.16.jpg', './train_vega\\Ankh.17.jpg', './train_vega\\Ankh.18.jpg', './train_vega\\Ankh.19.jpg', './train_vega\\Ankh.20.jpg', './train_vega\\Ankh.21.jpg', './train_vega\\bouche.01.jpg', './train_vega\\bouche.02.jpg', './train_vega\\bouche.03.jpg', './train_vega\\bouche.04.jpg', './train_vega\\bouche.05.jpg', './train_vega\\bouche.06.jpg', './train_vega\\bouche.07.jpg', './train_vega\\bouche.08.jpg', './train_vega\\bouche.09.jpg', './train_vega\\bouche.10.jpg', './train_vega\\bouche.11.jpg', './train_vega\\bouche

In [6]:
# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
	# load the image and extract the class label (assuming that our
	# path as the format: /path/to/dataset/{class}.{image_num}.jpg
	image = cv2.imread(imagePath)
	image_flip = cv2.flip(image,1)
	label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
	# extract raw pixel intensity "features", followed by a color
	# histogram to characterize the color distribution of the pixels
	# in the image
	pixels = image_to_feature_vector(image)
	pixels_flip = image_to_feature_vector(image_flip)
	hist = extract_color_histogram(image)
	hist_flip = extract_color_histogram(image_flip)

	# update the raw images, features, and labels matricies,
	# respectively
	rawImages = np.array(pixels)
	rawImages = np.array(pixels_flip)
	features = np.array(hist)
	features = np.array(hist_flip)
	labels = np.array(label)
	labels = np.array(label)

	# show an update every 1,000 images
	if i > 0 and i % 1000 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))
      
rawImages = np.array(rawImages)
features = np.array(features)
labels = np.array(labels)
# partition the data into training and testing splits, using 75%
# of the data for training and the remaining 25% for testing
(trainRI, testRI, trainRL, testRL) = train_test_split(
	rawImages, labels, test_size=0.25, random_state=42)
(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
	features, labels, test_size=0.25, random_state=42)

TypeError: Singleton array array('skara', 
      dtype='<U5') cannot be considered a valid collection.

In [11]:
# train and evaluate a SVM classifer on the raw pixel intensities
print("[INFO] evaluating raw pixel accuracy...")
model =  SVC(kernel='linear', C=1, random_state=0)
model.fit(trainRI, trainRL)
acc = model.score(testRI, testRL)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating raw pixel accuracy...
[INFO] raw pixel accuracy: 26.79%


In [19]:
# train and evaluate a SVM classifer on the histogram
# representations
print("[INFO] evaluating histogram accuracy...")
model =  SVC()
model.fit(trainFeat, trainLabels)
acc = model.score(testFeat, testLabels)
print("[INFO] histogram accuracy: {:.2f}%".format(acc * 100))

[INFO] evaluating histogram accuracy...
[INFO] histogram accuracy: 66.67%
